In [4]:
import itertools
import typing

import numpy as np

In [5]:
# helper functions
def get_amp_output(ic: np.ndarray, comms: typing.Tuple[int, int]) -> list:
    '''Returns amp output for phase setting and signal: (setting, sig).'''
    ix = 0
    id_ix = 0
    diag_codes = []

    while ix < len(ic):
        pars = f"{ic[ix]:04}"
        opcode = int(pars[2:]) 
            
        if opcode == 3:
            ic[ic[ix+1]] = comms[id_ix]
            id_ix += 1
            ix += 2
        elif opcode == 4:
            diag_codes.append(ic[ic[ix+1]])
            comms.append(ic[ic[ix+1]])
            ix += 2            
        elif opcode == 99:
            break
            
        else:
            val_1 = ic[ic[ix+1]] if int(pars[1]) == 0 else ic[ix+1]
            val_2 = ic[ic[ix+2]] if int(pars[0]) == 0 else ic[ix+2]
            
            if opcode == 1: 
                ic[ic[ix+3]] = val_1 + val_2
            if opcode == 2:
                ic[ic[ix+3]] = val_1 * val_2
            if opcode == 7:
                ic[ic[ix+3]] = 1 if val_1 < val_2 else 0
            if opcode == 8:
                ic[ic[ix+3]] = 1 if val_1 == val_2 else 0
             
            if opcode == 5:
                ix = val_2 if val_1 != 0 else ix + 3
            elif opcode == 6:
                ix = val_2 if val_1 == 0 else ix + 3
            else:
                ix += 4
        
    return diag_codes

def get_amp_loop_output(ic: np.ndarray, ic_ix: int, comms: typing.List[int], 
                        comms_ix: int) -> typing.Tuple[list, int, bool]:
    '''Returns amp output, index of pause, and whether the opcode '99' 
    was performed. Requires intcode, start index for intcode, list of 
    commands, and what command index to start on'''
    ix = ic_ix
    id_ix = comms_ix
    diag_codes = []
    code_99 = False
    
    while ix < len(ic):
        pars = f"{ic[ix]:04}"
        opcode = int(pars[2:]) 
            
        if opcode == 3:
            ic[ic[ix+1]] = comms[id_ix]
            id_ix += 1
            ix += 2
        elif opcode == 4:
            diag_codes.append(ic[ic[ix+1]])
            ix += 2
            break
        elif opcode == 99:
            code_99 = True
            break
            
        else:
            val_1 = ic[ic[ix+1]] if int(pars[1]) == 0 else ic[ix+1]
            val_2 = ic[ic[ix+2]] if int(pars[0]) == 0 else ic[ix+2]
            
            if opcode == 1: 
                ic[ic[ix+3]] = val_1 + val_2
            if opcode == 2:
                ic[ic[ix+3]] = val_1 * val_2
            if opcode == 7:
                ic[ic[ix+3]] = 1 if val_1 < val_2 else 0
            if opcode == 8:
                ic[ic[ix+3]] = 1 if val_1 == val_2 else 0
             
            if opcode == 5:
                ix = val_2 if val_1 != 0 else ix + 3
            elif opcode == 6:
                ix = val_2 if val_1 == 0 else ix + 3
            else:
                ix += 4
        
    return diag_codes, ix, code_99

In [3]:
%cd ../..

/Users/mwtmurphy/projects/advent-of-code


/Users/mwtmurphy/Library/Caches/pypoetry/virtualenvs/advent-of-code-rjxGAg1e-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
with open("data/2019/day_7.txt") as infile:
    intcode = np.array(infile.read().split(","), dtype=int)

    phases = list(itertools.permutations(range(5)))
    out_thrusts = []

    for seq in phases:
        in_sig = 0
        intcode_copy = intcode.copy()
        
        for in_code in seq:
            amp_output = get_amp_output(intcode_copy, [in_code, in_sig])
            in_sig = amp_output[0]
            
        out_thrusts.append(amp_output[0])

    ans_1 = max(out_thrusts)

    phases = list(itertools.permutations(range(5, 10)))
    out_thrusts = []

    for seq in phases:
        in_sig = 0
        com_ix = 0
        code_99 = False
        
        state = {}
        for i in range(5, 10):
            state[i] = {"ix": 0, "ic": intcode.copy()}
        
        while not code_99:
            for in_code in seq:
                amp_output, state[in_code]["ix"], code_99 = get_amp_loop_output(
                    state[in_code]["ic"], state[in_code]["ix"], [in_code, in_sig], com_ix
                )
                
                if amp_output:
                    in_sig = amp_output[0]
                else:
                    break
                
            if not com_ix:
                com_ix = 1
            
        out_thrusts.append(in_sig)

    ans_2 = max(out_thrusts)

    print("** Day 7 **")
    print(f"Highest signal that can be sent for problem 7.1: {ans_1}")
    print(f"Highest signal that can be sent for problem 7.2: {ans_2}", end="\n\n")

** Day 7 **
Highest signal that can be sent for problem 7.1: 440880
Highest signal that can be sent for problem 7.2: 3745599

